<a href="https://colab.research.google.com/github/wild10/SemEval_2019_OffensiveLanguage/blob/master/SemEval_2019_BERT_and_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

primero vamos a **instalar BERT** y sus dependencias cada vez que iniciamos sesion

In [0]:
!pip install pytorch-pretrained-bert

     |████████████████████████████████| 133kB 4.8MB/s 
     |████████████████████████████████| 655kB 38.0MB/s 
  Stored in directory: /root/.cache/pip/wheels/35/e4/80/abf3b33ba89cf65cd262af8a22a5a999cc28fbfabea6b38473
Successfully built regex


Offensive Language Identification using BERT and SVM

In [0]:
#primero vamos a montar el google drive

from google.colab import drive

# montar el driver de accesso
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import csv
import os
import logging
import argparse
import random
from tqdm import tqdm, trange

import pandas as pd
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler

from pytorch_pretrained_bert.tokenization import BertTokenizer # tokenizador del bert
from pytorch_pretrained_bert.modeling import BertForSequenceClassification 
from pytorch_pretrained_bert.optimization import BertAdam  # obtimizador
from pytorch_pretrained_bert.file_utils import PYTORCH_PRETRAINED_BERT_CACHE

In [0]:
logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.INFO)
logger = logging.getLogger(__name__)


class InputExample(object):
    """Un solo training/test para una simple secuancia de clasificacion"""

    def __init__(self, guid, text_a, text_b=None, label=None):
        """Constructor de un InputExample.
        Args:
            guid: id para el ejemplo
            text_a: string. texto no tokenizado de la 1° secuencia.
            text_b: (Opcional) string. txt no tokenizado de la 2da secuencia.
            (para pares de secuencias).
            label: (Optional) cadena. La etiqueta del ejemplo. Esto debe ser
             especificados para entrenamientos y ejemplos, pero no para pruebas de prueba.
        """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label


class InputFeatures(object):
    """simple conjunto de caracteristicas."""

    def __init__(self, input_ids, input_mask, segment_ids, label_id):
        self.input_ids = input_ids
        self.input_mask = input_mask # mascara del BERT
        self.segment_ids = segment_ids
        self.label_id = label_id


class DataProcessor(object):
    """Base de datos para convertidores de datos para la clasificación de fechas."""

    def get_train_examples(self, data_dir):
        """Gets coleccion de `InputExample`s para el train set."""
        raise NotImplementedError()

    def get_dev_examples(self, data_dir):
        """Gets coleccion de `InputExample`s para el devset."""
        raise NotImplementedError()

    def get_labels(self):
        """Gets lista de labels para el dataset."""
        raise NotImplementedError()

    @classmethod
    def _read_tsv(cls, input_file, quotechar=None):
        """Lee un archivo de valores separados por tabulaciones."""
        with open(input_file, "r", encoding='utf-8') as f:
            reader = csv.reader(f, delimiter="\t", quotechar=quotechar)
            lines = []
            for line in reader:
                lines.append(line)
            return lines


class Offen_1_Processor(DataProcessor):
    def get_train_examples(self, data_dir):
        """ver clase base."""
        return self._create_trn_examples(
            self._read_tsv(os.path.join(data_dir, "offenseval-training-v1.tsv")), "train")

    def get_dev_examples(self, data_dir):
        """ver clase base."""
#        return self._create_test_examples(
#            self._read_tsv(os.path.join(data_dir, "testset-taska.tsv")), "dev")
        return self._create_dev_examples(
            self._read_tsv(os.path.join(data_dir, "offenseval-trial.txt")), "dev")

    def get_labels(self):
        """ver clase base."""
        return ["0", "1"]

    def _create_trn_examples(self, lines, set_type):
        """ creamos ejemplo para el training y dev sets."""
        examples = []
        for (i, line) in enumerate(lines):
#            guid = "%s-%s" % (set_type, i)
            guid = line[0]
            text_a = line[1]
            if line[2] == "OFF":
                label = '1'
            else:
                label = '0'
            examples.append(
                InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
        return examples

    def _create_dev_examples(self, lines, set_type):
        """creamos ejemplo para el training y dev sets."""
        examples = []
        for (i, line) in enumerate(lines):
            guid = "%s-%s" % (set_type, i)
            text_a = line[0]
            if line[1] == "OFF":
                label = '1'
            else:
                label = '0'
            examples.append(
                InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
        return examples

    def _create_test_examples(self,lines,set_type):
        examples = []
        for (i, line) in enumerate(lines):
            if i != 0:
    #            guid = "%s-%s" % (set_type, i)
                guid = line[0]
                text_a = line[1]
                label = '0'
                examples.append(
                    InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
        return examples

class Offen_2_Processor(DataProcessor):
    def get_train_examples(self, data_dir):
        """ver clase base."""
        return self._create_trn_examples(
            self._read_tsv(os.path.join(data_dir, "offenseval-training-v1.tsv")), "train")

    def get_dev_examples(self, data_dir):
        """ver clase base."""
        return self._create_dev_examples(
            self._read_tsv(os.path.join(data_dir, "offenseval-trial.txt")), "dev")

    def get_labels(self):
        """ver clase base."""
        return ["0", "1"]

    def _create_trn_examples(self, lines, set_type):
        # creamos...
        examples = []
        for (i, line) in enumerate(lines):
            if line[3] != "subtask_b" and line[3] != "NULL":
                guid = "%s-%s" % (set_type, i)
                text_a = line[1]
                if line[3] == "TIN":
                    label = '1'
                elif line[3] == "UNT":
                    label = '0'
                examples.append(
                    InputExample(guid=guid, text_a=text_a, text_b=None, label=label))

        return examples

    def _create_dev_examples(self, lines, set_type):
        # creamos...
        examples = []
        for (i, line) in enumerate(lines):
            if line[2] != "subtask_b" and line[2] != "NULL":
                guid = "%s-%s" % (set_type, i)
                text_a = line[0]
                if line[2] == "TIN":
                    label = '1'
                elif line[2] == "UNT":
                    label = '0'
                examples.append(
                    InputExample(guid=guid, text_a=text_a, text_b=None, label=label))

        return examples



def convert_examples_to_features(examples, label_list, max_seq_length, tokenizer):
    """ cargar data en una lista de `InputBatch`s."""

    label_map = {label : i for i, label in enumerate(label_list)}

    features = []
    for (ex_index, example) in enumerate(examples):
        tokens_a = tokenizer.tokenize(example.text_a)

        tokens_b = None
        if example.text_b:
            tokens_b = tokenizer.tokenize(example.text_b)
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > max_seq_length - 2:
                tokens_a = tokens_a[:(max_seq_length - 2)]

        # The convention in BERT is:
        # (a) For sequence pairs:
        #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
        #  type_ids: 0   0  0    0    0     0       0 0    1  1  1  1   1 1
        # (b) For single sequences:
        #  tokens:   [CLS] the dog is hairy . [SEP]
        #  type_ids: 0   0   0   0  0     0 0
        #
        # Where "type_ids" are used to indicate whether this is the first
        # sequence or the second sequence. The embedding vectors for `type=0` and
        # `type=1` were learned during pre-training and are added to the wordpiece
        # embedding vector (and position vector). This is not *strictly* necessary
        # since the [SEP] token unambigiously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.
        #
        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        tokens = ["[CLS]"] + tokens_a + ["[SEP]"]
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens += tokens_b + ["[SEP]"]
            segment_ids += [1] * (len(tokens_b) + 1)

        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        input_mask = [1] * len(input_ids)

        # Zero-pad up to the sequence length.
        padding = [0] * (max_seq_length - len(input_ids))
        input_ids += padding
        input_mask += padding
        segment_ids += padding

        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length

        label_id = label_map[example.label]
        if ex_index < 5:
            logger.info("*** Example ***")
            logger.info("guid: %s" % (example.guid))
            logger.info("tokens: %s" % " ".join(
                    [str(x) for x in tokens]))
            logger.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
            logger.info("input_mask: %s" % " ".join([str(x) for x in input_mask]))
            logger.info(
                    "segment_ids: %s" % " ".join([str(x) for x in segment_ids]))
            logger.info("label: %s (id = %d)" % (example.label, label_id))

        features.append(
                InputFeatures(input_ids=input_ids,
                              input_mask=input_mask,
                              segment_ids=segment_ids,
                              label_id=label_id))
    return features


def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """truncamos un par de secuencias a la longitud maxima"""

    """
    Esta es una heurística simple que siempre truncará la secuencia
    más larga un token a la vez. Esto tiene más sentido que truncar un
    porcentaje igual de tokens de cada una, ya que si una secuencia es
    muy corta, es probable que cada token truncado contenga más información
    que una secuencia más larga.
    """
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()

def accuracy(out, labels):
    outputs = np.argmax(out, axis=1)
    return np.sum(outputs == labels)

def warmup_linear(x, warmup=0.002):
    if x < warmup:
        return x/warmup
    return 1.0 - x


instalar apex en python 

In [0]:
!git clone https://github.com/NVIDIA/apex

Cloning into 'apex'...
remote: Enumerating objects: 57, done.
remote: Counting objects: 100% (57/57), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 4693 (delta 26), reused 35 (delta 18), pack-reused 4636
Receiving objects: 100% (4693/4693), 8.73 MiB | 20.00 MiB/s, done.
Resolving deltas: 100% (3038/3038), done.


In [0]:
!cd apex && pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" .

/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/install.py:244: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-jtyioubu
Created temporary directory: /tmp/pip-req-tracker-kubwz3ux
Created requirements tracker '/tmp/pip-req-tracker-kubwz3ux'
Created temporary directory: /tmp/pip-install-rcpjgniv
Processing /content/apex
  Created temporary directory: /tmp/pip-req-build-hdnvttpw
  Added file:///content/apex to build tracker '/tmp/pip-req-tracker-kubwz3ux'
    Running setup.py (path:/tmp/pip-req-build-hdnvttpw/setup.py) egg_info for package from file:///content/apex
    Running command python setup.py egg_info

     If your intention is to cross-compile, this is not an error.

    torch.__version__  =  1.1.0
    running egg_info
    creating pip-egg-info/apex.egg-info
    writing pip-egg-info/apex.egg-info

In [0]:
import torch
print(torch.__version__)

1.1.0


Main llamada principal a las funciones

In [0]:
def main(): 
  # parametros principales
  data_dir = "/gdrive/My Drive/SemEval2019/training"
  bert_model = "bert-base-uncased"
  task_name = "one"
  output_dir = "/gdrive/My Drive/SemEval2019/output/"
  # otroa parametros
  max_seq_length = 80
  do_train = True
  do_eval = True
  do_lower_case = True
  train_batch_size = 32
  learning_rate = 5e-5
  num_train_epochs = 2
  print("task_name: ",task_name)
  print("do_train: ", do_train)
  print("do_eval: ", do_eval)
  print("do_lower_case: ", do_lower_case)
  print("data_dir: ", data_dir)
  print("bert_model: ",bert_model)
  print("max_seq_length: ", max_seq_length)
  print("train_batch_size: ", train_batch_size)
  print("learning_rate: ", learning_rate)
  print("num_train_epochs: ", num_train_epochs)
  print("output_dir: ", output_dir)

  processors = {
        "one": Offen_1_Processor,
        "two": Offen_2_Processor
    }

  num_labels_task = {
        "one": 2,
        "two": 2
  }
  # parametros adicionales defaul
  local_rank = -1
  no_cuda = True
  
  fp16 = False #True
  gradient_accumulation_steps = 1

  '''
  if local_rank == -1 or no_cuda:
    device = torch.device("cuda" if torch.cuda.is_available() and not no_cuda else "cpu")
    n_gpu = torch.cuda.device_count()
  else:
  	torch.cuda.set_device(local_rank)
    device = torch.device("cuda",local_rank)
    n_gpu = 1
    # Initializes the distributed backend which will take care of sychronizing nodes/GPUs
    torch.distributed.init_process_group(backend='nccl')
   '''
  if local_rank == -1 or no_cuda:
        device = torch.device("cuda" if torch.cuda.is_available() and not no_cuda else "cpu")
        n_gpu = torch.cuda.device_count()
  else:
      torch.cuda.set_device(local_rank)
      device = torch.device("cuda", local_rank)
      n_gpu = 1
      # Initializes the distributed backend which will take care of sychronizing nodes/GPUs
      torch.distributed.init_process_group(backend='nccl')
    
  logger.info("device: {} n_gpu: {}, distributed training: {}, 16-bits training: {}".format(
        device, n_gpu, bool(local_rank != -1), fp16))

  if gradient_accumulation_steps < 1:
        raise ValueError("Invalid gradient_accumulation_steps parameter: {}, should be >= 1".format(
                            gradient_accumulation_steps))
  # gradient_accumulation_steps = 1
  train_batch_size = int(train_batch_size / gradient_accumulation_steps)

  #aqui empieza la configuracion real
  seed = 42
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  if n_gpu > 0:
  	torch.cuda.manual_seed_all(seed)

  if not do_train and not do_eval:
  	raise ValueError("por lo menos 'do_train' o 'do_eval' deben ser True")
  #if os.path..

  # task name en minuscula
  task_name = task_name.lower()
  # validacion de tarea
  if task_name not in processors:
    raise ValueError("Tarea no encontrada : %s" % (task_name))

  #print("processor: ", processors[task_name]())
  #print("num_labels: ", num_labels_task[task_name])
  processor = processors[task_name]()
  num_labels = num_labels_task[task_name]
  label_list = processor.get_labels() 
  print("prin label_list: ", label_list)
  
  tokenizer = BertTokenizer.from_pretrained(bert_model, do_lower_case=do_lower_case)

  train_examples = None
  num_train_steps = None
  if do_train:
      train_examples = processor.get_train_examples(data_dir)
      num_train_steps = int(
          len(train_examples) / train_batch_size / gradient_accumulation_steps * num_train_epochs)
  
  # Prepare model
  model = BertForSequenceClassification.from_pretrained(bert_model,
#              cache_dir=PYTORCH_PRETRAINED_BERT_CACHE / 'distributed_{}'.format(args.local_rank),
            num_labels = num_labels)
  if fp16:
      model.half()
  model.to(device)
  if local_rank != -1:
      try:
          from apex.parallel import DistributedDataParallel as DDP
      except ImportError:
          raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use distributed and fp16 training.")
          
      model = DDP(model)
  elif n_gpu > 1:
    model = torch.nn.DataParallel(model)
  
  # Prepare optimizer
  param_optimizer = list(model.named_parameters())
  no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
  optimizer_grouped_parameters = [
      {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
      {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
      ]
  t_total = num_train_steps
  # parametro extra
  loss_scale = 0
  warmup_proportion = 0.1
  eval_batch_size = 8
  
  if local_rank != -1:
      t_total = t_total // torch.distributed.get_world_size()
      
  if fp16:
      try:
          from apex.optimizers import FP16_Optimizer
          from apex.optimizers import FusedAdam
      except ImportError:
          raise ImportError("Please instala apex from https://www.github.com/nvidia/apex to use distributed and fp16 training.")

      optimizer = FusedAdam(optimizer_grouped_parameters,
                            lr=learning_rate,
                            bias_correction=False,
                            max_grad_norm=1.0)
      if loss_scale == 0:
          optimizer = FP16_Optimizer(optimizer, dynamic_loss_scale=True)
      else:
          optimizer = FP16_Optimizer(optimizer, static_loss_scale=loss_scale)

  else:
      optimizer = BertAdam(optimizer_grouped_parameters,
                           lr=learning_rate,
                           warmup=warmup_proportion,
                           t_total=t_total)
  global_step = 0
  nb_tr_steps = 0
  tr_loss = 0
  if do_train:
      train_features = convert_examples_to_features(
          train_examples, label_list, max_seq_length, tokenizer)
      logger.info("***** Running training *****")
      logger.info("  Num examples = %d", len(train_examples))
      logger.info("  Batch size = %d", train_batch_size)
      logger.info("  Num steps = %d", num_train_steps)
      all_input_ids = torch.tensor([f.input_ids for f in train_features], dtype=torch.long)
      all_input_mask = torch.tensor([f.input_mask for f in train_features], dtype=torch.long)
      all_segment_ids = torch.tensor([f.segment_ids for f in train_features], dtype=torch.long)
      all_label_ids = torch.tensor([f.label_id for f in train_features], dtype=torch.long)
      train_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
      if local_rank == -1:
          train_sampler = RandomSampler(train_data)
      else:
          train_sampler = DistributedSampler(train_data)
      train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=train_batch_size)

      model.train()
      
      for _ in trange(int(num_train_epochs), desc="Epoch"):
          tr_loss = 0
          nb_tr_examples, nb_tr_steps = 0, 0
          for step, batch in enumerate(tqdm(train_dataloader, desc="Iteration")):
              batch = tuple(t.to(device) for t in batch)
              input_ids, input_mask, segment_ids, label_ids = batch
              loss = model(input_ids, segment_ids, input_mask, label_ids)
              if n_gpu > 1:
                  loss = loss.mean() # mean() to average on multi-gpu.
              if gradient_accumulation_steps > 1:
                  loss = loss / gradient_accumulation_step
              if fp16:
                  optimizer.backward(loss)
              else:
                  loss.backward()  
              tr_loss += loss.item()
              nb_tr_examples += input_ids.size(0)
              nb_tr_steps += 1
              if (step + 1) % gradient_accumulation_steps == 0:
                  # modificar el learning rate con un especial uso warm up de BERT
                  lr_this_step = learning_rate * warmup_linear(global_step/t_total, warmup_proportion)
                  for param_group in optimizer.param_groups:
                      param_group['lr'] = lr_this_step
                  optimizer.step()
                  optimizer.zero_grad()
                  global_step += 1
   
  # Save a trained model
  model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self
  output_model_file = os.path.join(output_dir, "pytorch_model.bin")
  if do_train:
      torch.save(model_to_save.state_dict(), output_model_file)
  
  # Load a trained model that you have fine-tuned
  model_state_dict = torch.load(output_model_file)
  model = BertForSequenceClassification.from_pretrained(bert_model, state_dict=model_state_dict, num_labels=num_labels)
  model.to(device)
  
  if do_eval and (local_rank == -1 or torch.distributed.get_rank() == 0):
      eval_examples = processor.get_dev_examples(data_dir)
      eval_features = convert_examples_to_features(
          eval_examples, label_list, max_seq_length, tokenizer)
      logger.info("***** Running evaluation *****")
      logger.info("  Num examples = %d", len(eval_examples))
      logger.info("  Batch size = %d", eval_batch_size)
      all_input_ids = torch.tensor([f.input_ids for f in eval_features], dtype=torch.long)
      all_input_mask = torch.tensor([f.input_mask for f in eval_features], dtype=torch.long)
      all_segment_ids = torch.tensor([f.segment_ids for f in eval_features], dtype=torch.long)
      all_label_ids = torch.tensor([f.label_id for f in eval_features], dtype=torch.long)
      eval_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
      # Run prediction for full data
      eval_sampler = SequentialSampler(eval_data)
      eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=eval_batch_size)

      model.eval()
      eval_loss, eval_accuracy = 0, 0
      nb_eval_steps, nb_eval_examples = 0, 0

      for input_ids, input_mask, segment_ids, label_ids in tqdm(eval_dataloader, desc="Evaluating"):
          input_ids = input_ids.to(device)
          input_mask = input_mask.to(device)
          segment_ids = segment_ids.to(device)
          label_ids = label_ids.to(device)

          with torch.no_grad():
              tmp_eval_loss = model(input_ids, segment_ids, input_mask, label_ids)
              logits = model(input_ids, segment_ids, input_mask)

          logits = logits.detach().cpu().numpy()
          label_ids = label_ids.to('cpu').numpy()
          tmp_eval_accuracy = accuracy(logits, label_ids)

          eval_loss += tmp_eval_loss.mean().item()
          eval_accuracy += tmp_eval_accuracy

          nb_eval_examples += input_ids.size(0)
          nb_eval_steps += 1

      eval_loss = eval_loss / nb_eval_steps
      eval_accuracy = eval_accuracy / nb_eval_examples
      loss = tr_loss/nb_tr_steps if do_train else None
      result = {'eval_loss': eval_loss,
                'eval_accuracy': eval_accuracy,
                'global_step': global_step,
                'loss': loss}

#     input_ids = pd.Series(np.squeeze(input_ids.detach().cpu().numpy()))
#     pred = pd.Series(np.squeeze(np.argmax(logits, axis=1)))
#     sub = pd.DataFrame({"id":input_ids,"pred":pred})
#     sub.to_csv(os.path.join(args.output_dir, "submission.csv"),header=None,index=False)
      print(np.squeeze(input_ids.detach().cpu().numpy()).shape)
      print(logits.shape)
      output_eval_file = os.path.join(output_dir, "eval_results.txt")
      with open(output_eval_file, "w") as writer:
          logger.info("***** Eval results *****")
          for key in sorted(result.keys()):
              logger.info("  %s = %s", key, str(result[key]))
              writer.write("%s = %s\n" % (key, str(result[key])))
              
main() # llamado al main principal

06/14/2019 16:18:04 - INFO - __main__ -   device: cpu n_gpu: 0, distributed training: False, 16-bits training: False


task_name:  one
do_train:  True
do_eval:  True
do_lower_case:  True
data_dir:  /gdrive/My Drive/SemEval2019/training
bert_model:  bert-base-uncased
max_seq_length:  80
train_batch_size:  32
learning_rate:  5e-05
num_train_epochs:  2
output_dir:  /gdrive/My Drive/SemEval2019/output/
prin label_list:  ['0', '1']


06/14/2019 16:18:05 - INFO - pytorch_pretrained_bert.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt not found in cache, downloading to /tmp/tmp_phtsf9f
100%|██████████| 231508/231508 [00:00<00:00, 1224980.30B/s]
06/14/2019 16:18:05 - INFO - pytorch_pretrained_bert.file_utils -   copying /tmp/tmp_phtsf9f to cache at /root/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
06/14/2019 16:18:05 - INFO - pytorch_pretrained_bert.file_utils -   creating metadata file for /root/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
06/14/2019 16:18:05 - INFO - pytorch_pretrained_bert.file_utils -   removing temp file /tmp/tmp_phtsf9f
06/14/2019 16:18:05 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws

(8, 80)
(8, 2)
